ChunModelling - ANN

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model
import tensorflow as tf

In [ ]:
%cd '/content/gdrive/My Drive/.../'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
df = pd.read_csv("Churn_Modelling.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.groupby(by='Geography').count()

In [ ]:
X = df.iloc[:, 3:13].values # inputs
y = df.iloc[:, 13].values # output

In [ ]:
exited = np.count_nonzero(y)
exited

In [ ]:
no_exited = y.shape[0] - exited
no_exited

In [ ]:
from scipy.stats import itemfreq
# khach hang o lai/ra di
itemfreq(y)

In [ ]:
# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])],
remainder='passthrough')
X = ct.fit_transform(X.tolist())
# save 3 model nay pickle: labelencoder_X_1, labelencoder_X_2, ct
# Save the Modle to file in the current working directory
Pkl_Filename = "Pickle_RL_Model.pkl"
with open(Pkl_Filename, 'wb') as file:
pickle.dump(Model, file)
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:
model_load = pickle.load(file)

In [ ]:
# scale truoc => split sau
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)
import pickle
scalerfile = 'sc.sav'
pickle.dump(sc, open(scalerfile, 'wb'))

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 0)

In [ ]:
X_train[:5]

In [ ]:
X_train[0].shape

Build Model ANN

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential

In [ ]:
# Intitialing the ANN
classifier = Sequential()

In [ ]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, activation='relu', input_shape=(11,)))
classifier.add(Dropout(rate=0.1))

In [ ]:
# Addding the second hidden layer
# output of previous layers => input of the next layers
classifier.add(Dense(units = 6,
                      activation='relu'))
classifier.add(Dropout(rate=0.1))

In [ ]:
# Adding the output layer
classifier.add(Dense(units = 1,
                    activation='sigmoid')) # like logistic regression

In [ ]:
#compiling ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
classifier.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [ ]:
plot_model(classifier, to_file='ANN_model.png', show_shapes=True)
Image(filename='ANN_model.png')

Calculate params: fully connected: Param # = input * output +
output

dense_input: Input layer: shape(None, 11), Param #=0

dense:Dense: shape(?, 6), Param # = input * output + output = 11 * 6 + 6 = 72

dense_1:Dense: shape(?, 6), Param # = input * output + output = 6 * 6 + 6 = 42

dense_2:Dense: shape(?, 6), Param # = input * output + output = 6 * 1 + 1 = 7

In [ ]:
# Fitting classifier to the Training set
history = classifier.fit(X_train, y_train,
                          epochs = 100,
                          batch_size=64,
                          validation_data=(X_test, y_test))

In [ ]:
# Fitting classifier to the Training set
history = classifier.fit(X_train, y_train,
epochs = 100,
batch_size=64,
validation_data=(X_test, y_test))

In [ ]:
plt.figure()
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_accuracy'])
plt.title('val Loss & val Accuracy')
plt.ylabel('Values')
plt.xlabel('Epoch')
plt.legend(['val Loss', 'val Acc'])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
y_pred[:5]

In [ ]:
y_pred = y_pred >= 0.5
print(y_pred)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
acc = (cm[0][0] + cm[1][1])/ (cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
print(acc)

In [ ]:
# Evaluation
print("Accuracy:", classifier.evaluate(X_test, y_test))

In [ ]:
#Save the result
from tensorflow.keras.models import load_model
# Creates a HDF5 file 'my_model.h5'
classifier.save('ANN_model.h5')

Use our ANN model to predict if the customer with the following informations will leave the

bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: 50000

So should we say goodbye to that customer ?

In [ ]:
new_pred = classifier.predict(sc.transform(np.array([[0, 0, 600, 1, 40, 3, 60000,
                                                        2, 1, 1, 50000]])))

In [ ]:
new_pred = new_pred > 0.5
new_pred